# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Notebook Principal

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**http://fdelmazo.github.io/7506-Datos/**

**https://www.kaggle.com/datatouille2018/competitions**

Continuando la investigación sobre la empresa Trocafone realizada en el [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html), se busca determinar la probabilidad de que un usuario del sitio realice una conversión en el período determinado.

Notebooks en orden de corrida y lectura:

0. [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) --> Familiarización con el set de datos y exploración de estos.

1. [Investigación Previa](https://fdelmazo.github.io/7506-Datos/TP2/investigacion.html) --> Con ayuda de lo trabajado en el TP1, se averiguan más cosas de las datos, en busqueda de que poder reutilizar.

2. [Creación de Dataframes](https://fdelmazo.github.io/7506-Datos/TP2/new_dataframes.html) --> Como parte del feature engineering, se crean dataframes nuevos con información de los productos del sitio y de como se accede a este (marcas, sistemas operativos, etc).

3. [Feature Engineering](https://fdelmazo.github.io/7506-Datos/TP2/feature_engineering.html) --> Busqueda de atributos de los usuarios de los cuales se busca predecir la conversión.

4. [Submission Framework](https://fdelmazo.github.io/7506-Datos/TP2/submission_framework.html) --> Pequeño framework para construir las postulaciones de labels. 

5. [Parameter Tuning](https://fdelmazo.github.io/7506-Datos/TP2/parameter_tuning.html) --> Busqueda de los mejores hiper-parametros para cada algoritmo de ML.

6. [Feature Selection](https://fdelmazo.github.io/7506-Datos/TP2/feature_selection.html) --> Busqueda de la combinación de features más favorable.

7. TP2 (este notebook)--> Teniendo todo en cuenta, usando los dataframes con todos los atributos buscados y encontrados, se definen y aplican los algoritmos de clasificación, se realizan los entrenamientos y posteriores predicciones de conversiones y finalmente se arman las postulaciones de labels.

In [ ]:
# Set-up inicial, se deja comentado para evitar instalarle módulos al usuario
## Primero, descargar los datasets de no tenerlos

# Antes de comenzar, setear las credenciales (usuario y token)

# 1. Visitar: https://www.kaggle.com/datatouille2018/account (con la cuenta que sea)
# 2. Tocar en Create New API Token
# 3. Guardar el archivo descargado en ~/.kaggle/kaggle.json

# !pip install kaggle # https://github.com/Kaggle/kaggle-api
# !kaggle competitions download -c trocafone -p data
# !unzip -q data/events_up_to_01062018.csv.zip -d data
# !rm data/events_up_to_01062018.csv.zip
# !ls data/

## Luego, descargar los módulos a utilizar a lo largo de todo el trabajo

# !pip install nbimporter
# !conda install -c conda-forge xgboost 

In [ ]:
import nbimporter # pip install nbimporter
import pandas as pd
import numpy as np
import calendar
from parameter_tuning import get_hiper_params
from feature_selection import get_feature_selection
import submission_framework as SF

seed = 42
hiper_params = get_hiper_params()
feature_selection = get_feature_selection()

In [ ]:
df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

display(df_users.head(), df_y.head())

## Algoritmos de Machine Learning

In [ ]:
posibilidades_algoritmos = []

---

### Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

model_name = 'decision_tree'
params = hiper_params[model_name]
model = DecisionTreeClassifier(**params,random_state=seed)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_name = 'random_forest'
params = hiper_params[model_name]
model = RandomForestClassifier(**params,random_state=seed)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### XGBoost


In [ ]:
import xgboost as xgb #conda install -c conda-forge xgboost 

model_name = 'xgboost'
params = hiper_params[model_name]
model = xgb.XGBClassifier(**params,random_state=seed)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_name = 'knn'
params = hiper_params[model_name]
K = params['n_neighbors']
model_name = f'KNN{K}'

model = KNeighborsClassifier(**params)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

### Naive-Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB


model_name = 'naive_bayes'
params = hiper_params[model_name]
model = GaussianNB(**params)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

## Encontrando el mejor submit

Corremos todos los algoritmos definidos sobre esas combinaciones, en busqueda de su mejor combinación de hiper-parametros.

Finalmente, se corren todos los algoritmos en su mejor combinación contra todos los set de features definidos, en busqueda de la mejor fusión universal.

In [ ]:
columnas_a_mano = ['total_checkouts_month_5',
                    'timestamp_last_checkout',
                    'timestamp_last_event',
                    'has_checkout_month_5',
                    'total_checkouts',
                    'days_to_last_event',
                    'total_checkouts_last_week',
                    'total_checkouts_months_1_to_4',
                    'total_conversions',
                    'total_session_conversions',
                    'total_events',
                    'total_sessions',
                    'avg_events_per_session',
                    'total_session_checkouts',
                    'has_checkout'
                    ]

columnas_a_mano_2 = ['dow_last_conversion', 
                     'has_conversion_last_week', 'total_conversions_month_4', 
                     'total_session_checkouts', 'doy_last_conversion', 'timestamp_last_event', 
                     'dow_last_checkout', 'total_checkouts', 'has_checkout', 'doy_last_checkout', 
                     'has_checkout_month_1', 'timestamp_last_checkout', 'total_sessions', 
                     'woy_last_event', 'has_checkout_month_5', 'avg_events_per_session']

In [ ]:
posibilidades_features = {
    'Full Dataframe':None,
    'Best Cumulative Importance':feature_selection['best_features_progresivo'],
    'Best Forward Selection':feature_selection['best_features_forward'],
    'Best Backward Elimination':feature_selection['best_features_backward'],
    'Leap Cumulative Importance':feature_selection['features_con_saltos_progresivo'],
    'Leap Forward Selection':feature_selection['features_con_saltos_forward'],
    'Selección a Mano': columnas_a_mano,
    'Selección a Mano 2': columnas_a_mano_2
}

In [ ]:
max_auc = 0
campeon_nombre = ''
campeon_algoritmo = None
campeon_forma = None
campeon_features = None

es_ensamble = False

for nombre,algoritmo in posibilidades_algoritmos:
    for forma, features in posibilidades_features.items():
        model_with_name = (f'{forma} - {nombre}',algoritmo)
        model, auc = SF.full_framework_wrapper(df_users, df_y, model_with_name, columns=features)
        if auc > max_auc:
            max_auc = auc
            campeon_nombre = nombre
            campeon_algoritmo = algoritmo
            campeon_forma = forma
            campeon_features = features

In [ ]:
display(f"Mejor Apuesta (so far): {campeon_nombre} ({max_auc:.4f} AUC) - Features: {campeon_forma}")
display(f"Features: {campeon_features}")

### Ensambles de algoritmos

Se prueban todas las posibilidades de combinación de 2 algoritmos y se queda con la que arroje mejor AUC

In [ ]:
from itertools import combinations
                             
def obtain_combinations(n):
    result = list(combinations(posibilidades_algoritmos, n))
    return result

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

algorithm_combinations = obtain_combinations(2)

for combinacion_algoritmo in algorithm_combinations:
    for forma, features in posibilidades_features.items():
        print(f'{forma} - ',end='')
        model, auc = SF.full_framework_ensemble_wrapper(df_users, df_y, combinacion_algoritmo, columns=features)
        if auc > max_auc:
            es_ensamble=True
            max_auc = auc
            campeon_algoritmo = combinacion_algoritmo
            campeon_forma = forma
            campeon_features = features

In [ ]:
if es_ensamble: campeon_nombre = f"{'+'.join([x[0] for x in campeon_algoritmo])}"
display(f"Mejor Apuesta: {campeon_nombre} ({max_auc:.4f} AUC) - Features: {campeon_forma}")
display(f"Features: {campeon_features}")

## Corrida Final

Se corre entrenando con X (y no X_train) el submit final.

In [ ]:
n_ensamble = 300

if es_ensamble:
    campeon_model, campeon_auc, csv_name, campeon_message = SF.full_framework_ensemble_wrapper(df_users, 
                                                                                                df_y, 
                                                                                                campeon_algoritmo,
                                                                                                columns=features,
                                                                                                n_ensamble=n_ensamble,
                                                                                                submit=True,
                                                                                                verbosity=1,
                                                                                                all_in=True)
else:
    campeon_model, campeon_auc, csv_name, campeon_message = SF.full_framework_wrapper(df_users, 
                                                                                    df_y, 
                                                                                    (campeon_nombre,campeon_algoritmo),
                                                                                    columns=features,
                                                                                    n_ensamble=n_ensamble,
                                                                                    submit=True,
                                                                                    verbosity=1,
                                                                                    all_in=True)   

In [ ]:
## Descomentar y submitear!
## Ojo, solo correr una vez!!!

#!kaggle competitions submit -f {csv_name} -m "{campeon_message}" trocafone

print()
print('https://www.kaggle.com/c/trocafone/submissions?sortBy=date')
print('https://www.kaggle.com/c/trocafone/leaderboard')

---
---
## Algoritmos y Features

* Naive Bayes

* Perceprton

* SVM

* https://github.com/urielkelman/abracadata/tree/master/TP2

* https://github.com/MatiasReimondo/Datos


## Lista de cosas que se pueden hacer

* [ ] Identificar bias y varianza, ploteando error de set de entrenamiento y error de set de test en funcion de cantidad de datos en set de entrenamiento (mismo plot)

* [ ] Perturbar datos de entrada -> reducir overfitting

* [ ] Plotear AUC nuestra, AUC kaggle segun submission

* [ ] Catboost

* [ ] Reclamar 50 usd aws &#128544; 

* [ ] Clustering para nuevos features para entrenar